Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('nils-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

# password read from file

Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed

Reserve the required equipment and resources

In [ ]:
worker_name = "worker-08"

reservation_id = None

# Check if the worker is already leased
leaseslist = list_reservations(brief=True)
for lease in leaseslist:
    if lease["name"] == worker_name+"-lease":
        reservation_id = lease["reservation_id"]
        logger.info("Worker already leased with reservation id " + reservation_id)
        break

# If the worker is not leased, lease it
if not reservation_id:
    worker_lease = reserve(
        { "type":"device", "name":worker_name, "duration": { "days":1, "hours":0 } }
    )
    worker_reservation_id = worker_lease["reservations"][0]["id"]

Create networks

In [ ]:

# Create nils-edgenet for talker
try: 
    edge_net = chi.network.get_network("nils-edgenet")
    edge_net_subnet = chi.network.get_subnet("nils-edgenet-subnet")
    logger.info("nils-edgenet already exists.")
except:
    edge_net = chi.network.create_network("nils-edgenet")
    edge_net_subnet = chi.network.create_subnet(
        "nils-edgenet-subnet", 
        edge_net["id"], 
        "10.70.70.0/24", 
        gateway_ip="10.70.70.1", 
        enable_dhcp=False
    )
    logger.success("nils-edgenet is created.")

# Create nils-devicenet for listener
try:
    end_net = chi.network.get_network("nils-devicenet")
    end_net_subnet = chi.network.get_subnet("nils-devicenet-subnet")
    logger.info("nils-devicenet already exists.")
except:
    end_net = chi.network.create_network("nils-devicenet")
    end_net_subnet = chi.network.create_subnet(
        "nils-devicenet-subnet", 
        end_net["id"], 
        "10.42.3.0/24", 
        gateway_ip="10.42.3.1", 
        enable_dhcp=False
    )
    logger.success("nils-devicenet is created.")

Create routers

In [ ]:
try:
    chi.network.get_router("nils-router")
    logger.info("Router already exists.")
except:
    router = chi.network.create_router("nils-router", "public")
    chi.network.add_subnet_to_router(router["id"], edge_net_subnet["id"])
    chi.network.add_subnet_to_router(router["id"], end_net_subnet["id"])
    logger.success("Router created and subnets added.")

Project settings

In [ ]:
discovery_addr = "10.70.70.80"
talker_addr = "10.70.70.90"
listener_addr = "10.70.70.100"

Start discovery-server

In [ ]:
public_net = chi.network.get_network("serverpublic")

discovery_image_name = "nilsjor/ros-humble-turtlebot:discovery-v0.1"

discovery_env_vars = {
    "ROS_DISCOVERY_SERVER":"localhost:11811",
    # "DNS_IP":"1.1.1.1",
    # "GATEWAY_IP":"130.237.11.97",
    # "PASS":"turtlebot"
}

discovery_labels = {
    "networks.1.interface":"eno12419",
    "networks.1.ip":discovery_addr+"/24",
    "networks.1.routes":"10.42.3.0/24-10.70.70.1",
    # "networks.2.interface":"ens1",
    # "networks.2.ip":"130.237.11.126/27",
    # "networks.2.gateway":"130.237.11.97",
    # "capabilities.privileged":"true",
}

discovery_container_name = "discovery-server"

try:
    chi.container.destroy_container(discovery_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

discovery_container = chi.container.create_container(
    name = discovery_container_name,
    image = discovery_image_name,
    reservation_id = worker_reservation_id,
    environment = discovery_env_vars,
    nets = [
        { "network" : edge_net['id'] },
        # { "network" : public_net['id'] },
    ],
    labels = discovery_labels,
)
chi.container.wait_for_active(discovery_container_name)
logger.success("Container deployed and active.")

Start talker

In [ ]:
talker_image_name = "nilsjor/ros-humble-turtlebot:talker-v0.2"

talker_env_vars = {
    "ROS_DISCOVERY_SERVER":discovery_addr+":11811",
}

talker_labels = {
    "networks.1.interface":"eno12419",
    "networks.1.ip":talker_addr+"/24",
    "networks.1.routes":"10.42.3.0/24-10.70.70.1",
}

talker_container_name = "talker"

try:
    chi.container.destroy_container(talker_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

talker_container = chi.container.create_container(
    name = talker_container_name,
    image = talker_image_name,
    reservation_id = worker_reservation_id,
    environment = talker_env_vars,
    nets = [
        { "network" : edge_net['id'] },
    ],
    labels = talker_labels,
)

chi.container.wait_for_active(talker_container_name)
logger.success("Container deployed and active.")

Start listener

In [ ]:
listener_image_name = "nilsjor/ros-humble-turtlebot:listener-v0.2"

listener_env_vars = {
    "ROS_DISCOVERY_SERVER":discovery_addr+":11811",
}

listener_labels = {
    "networks.1.interface":"eno12419",
    "networks.1.ip":listener_addr+"/24",
    "networks.1.routes":"10.42.3.0/24-10.70.70.1",
}

listener_container_name = "listener"

try:
    chi.container.destroy_container(listener_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

listener_container = chi.container.create_container(
    name = listener_container_name,
    image = listener_image_name,
    reservation_id = worker_reservation_id,
    environment = listener_env_vars,
    nets = [
        { "network" : edge_net['id'] },
    ],
    labels = listener_labels,
)

chi.container.wait_for_active(listener_container_name)
logger.success("Container deployed and active.")

Teardown

In [ ]:
status = get_container_status(listener_container_name)
if status:
    chi.container.destroy_container(listener_container_name)
    wait_until_container_removed(listener_container_name)

status = get_container_status(talker_container_name)
if status:
    chi.container.destroy_container(talker_container_name)
    wait_until_container_removed(talker_container_name)

logger.info(f"stopped and removed all containers")

# find the router again
router = None
try:
    router = chi.network.get_router("nils-router")
except Exception as ex:
    logger.info(f"could not find nils-router.")

if router:
    # remove all routes from the router
    chi.network.remove_all_routes_from_router(router["id"])
    logger.success(f"removed all routers from router")

    # remove all subnets from the router
    subnets = chi.network.list_subnets()
    logger.info(f"checking all {len(subnets)} subnets.")
    for subnet in subnets:
        try:
            chi.network.remove_subnet_from_router(router["id"],subnet["id"])
        except Exception as ex:
            pass
    logger.success(f"removed all subnets from router")

    chi.network.delete_router(router["id"])
    logger.success(f"deleted the router")

edgenet = None
try:
    edgenet = chi.network.get_network("nils-edgenet")
except Exception as ex:
    logger.info(f"could not find nils-edgenet.")

if edgenet:
    chi.network.delete_network(edgenet["id"])
    logger.success(f"deleted the nils-edgenet")

endnet = None
try:
    endnet = chi.network.get_network("nils-devicenet")
except Exception as ex:
    logger.info(f"could not find nils-devicenet.")
    
if endnet:
    chi.network.delete_network(endnet["id"])
    logger.success(f"deleted the nils-devicenet")

unreserve_byid(worker_lease["id"])